# **Introduction to PyTorch**
## Image Processing and Computer Vision - Lab Session No. 4

Contacts:

- Prof. Giuseppe Lisanti -> giuseppe.lisanti@unibo.it
- Prof. Samuele Salti -> samuele.salti@unibo.it
- Alex Costanzino -> alex.costanzino@unibo.it
- Francesco Ballerini -> francesco.ballerini4@unibo.it

Based on the [UvA Deep Learning Course](https://uvadlc.github.io).

## Import dependencies

In [ ]:
import random
import time

import matplotlib.pyplot as plt
import numpy as np

from tqdm.notebook import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.optim import Adam
from torch.utils.data import random_split, DataLoader
from torchvision import transforms as T, datasets

## Runtime settings

For this hands-on session and the next one, we are going to train on GPUs. To request a GPU for you virtual machine in Colab, you should change your runtime type:

![](https://cdn.analyticsvidhya.com/wp-content/uploads/2020/03/uc14.png)

## PyTorch basics

As a prerequisite, we recommend to be familiar with the `numpy` package as most machine learning frameworks are based on very similar concepts. If you are not familiar with NumPy yet, don't worry: here is a [tutorial](https://numpy.org/devdocs/user/quickstart.html) to go through.

So, let's start with importing PyTorch and check its version:

In [ ]:
import torch
print("Using torch", torch.__version__)

### Reproducibility
When **developing and debugging** neural networks is desirable to have a deterministic behaviour (see *ablations*).

For this reason, we are going to disable all the sources of randomness. Please note that completely reproducible results are not guaranteed (more detailed information at this [page](https://pytorch.org/docs/stable/notes/randomness.html)).

Please note that the flag `cudnn.benchmark = False` disables the auto-tuner that selects the optimal set of algorithms for your hardware and usually leads to slower runtime.

In [ ]:
def fix_random(seed: int) -> None:
    """Fix all the possible sources of randomness.

    Args:
        seed: the seed to use.
    """
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)

    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

fix_random(115)

### Tensors

Tensors are the PyTorch equivalent to NumPy arrays, with the addition of also having support for GPU acceleration (more on that later).

The name "tensor" is a generalization of concepts of arrays/matrices. For instance, a vector is a 1-D tensor, and a matrix a 2-D tensor.

Most common functions you know from NumPy can be used on tensors as well. Actually, since NumPy arrays are so similar to tensors, we can convert most tensors to NumPy arrays (and back), although we don't need it too often.

#### Initialization

Let's first start by looking at different ways of creating a tensor. There are many possible options, the simplest one is to call `torch.Tensor` passing the desired shape as input argument:

In [ ]:
x = torch.Tensor(2, 3, 4)
print(x)

The function `torch.Tensor` allocates memory for the desired tensor, but reuses any values that have already been stored in the memory by default underlying processes (*uninitialized memory*).

To directly assign values to the tensor during initialization, there are many alternatives including:

* `torch.zeros`: creates a tensor filled with zeros
* `torch.ones`: creates a tensor filled with ones
* `torch.rand`: creates a tensor with random values uniformly sampled between 0 and 1
* `torch.randn`: creates a tensor with random values sampled from a normal distribution with mean 0 and variance 1
* `torch.arange`: creates a tensor containing the values $N,N+1,N+2,...,M$
* `torch.Tensor` (input list): creates a tensor from the list elements you provide

In [ ]:
# Create a tensor from a (nested) list
x = torch.Tensor([[1, 2], [3, 4]])
print(x)

In [ ]:
# Create a tensor with random values between 0 and 1 with the shape [2, 3, 4]
x = torch.rand(2, 3, 4)
print(x)
print(x.shape)

#### Tensor to NumPy, and NumPy to Tensor

PyTorch tensors can be converted to NumPy arrays, and NumPy arrays back to tensors. To transform a NumPy array into a tensor, we can use the function `torch.from_numpy`:

In [ ]:
np_arr = np.array([[1, 2], [3, 4]])
tensor = torch.from_numpy(np_arr)

print(f"NumPy array:\n{np_arr}")
print(f"\nPyTorch tensor:\n{tensor}")

To transform a PyTorch tensor back to a NumPy array, we can use the function `.numpy()` on tensors:

In [ ]:
tensor = torch.arange(4)
np_arr = tensor.numpy()

print(f"PyTorch tensor:\n{tensor}")
print(f"\nNumpy array:\n{np_arr}")

The conversion of tensors to NumPy requires the tensor to be on the CPU, and not the GPU (more on GPU support in a later section).

In case you have a tensor on GPU, you need to call `.cpu()` on the tensor beforehand. Hence, you get a line like `np_arr = tensor.cpu().numpy()`.

#### Basic operations


In [ ]:
x1 = torch.rand(2, 3)
x2 = torch.rand(2, 3)
y = x1 + x2

print(f"X1:\n{x1}")
print(f"X2:\n{x2}")
print(f"Y:\n{y}")

In-place operations are usually marked with a underscore postfix (e.g. `add_` instead of `add`):

In [ ]:
x1 = torch.rand(2, 3)
x2 = torch.rand(2, 3)
print(f"X1 (before):\n{x1}")
print(f"X2 (before):\n{x2}")

x2.add_(x1)
print(f"X1 (after):\n{x1}")
print(f"X2 (after):\n{x2}")

Another common operation aims at changing the shape of a tensor. A tensor of size (2,3) can be re-organized to any other shape with the same number of elements (e.g. a tensor of size (6) or (3,2)). In PyTorch, this operation is called `view`:

In [ ]:
x = torch.arange(6)
print(f"X:\n{x}")

In [ ]:
x = x.view(2, 3)
print(f"X:\n{x}")

In [ ]:
x = x.permute(1, 0)  # swap dimension 0 and 1
print(f"X:\n{x}")

### GPU support

In [ ]:
gpu_avail = torch.cuda.is_available()
print(f"Is the GPU available? {gpu_avail}")

In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(f"Device: {device}")

In [ ]:
x = torch.zeros(2, 3)
x = x.to(device)
print(f"X:\n{x}")

In case you have a GPU, you should now see the attribute `device='cuda:0'` being printed next to your tensor. The `0` next to `cuda` indicates that this is the `0`-th GPU device on your computer. PyTorch also supports multi-GPU systems, but you will only need this feature once you have very big networks to train (if interested, see the [PyTorch documentation](https://pytorch.org/docs/stable/distributed.html#distributed-basics)).

We can also compare the runtime of a large matrix multiplication on the CPU with the same operation on the GPU:

In [ ]:
x = torch.randn(5000, 5000)

## CPU version
start_time = time.time()
_ = torch.matmul(x, x)
end_time = time.time()
print(f"CPU time: {(end_time - start_time):6.5f}s")

## GPU version
x = x.to(device)

_ = torch.matmul(x, x)  # first operation to 'burn in' GPU

# CUDA is asynchronous, so we need to use different timing functions
start = torch.cuda.Event(enable_timing=True)
end = torch.cuda.Event(enable_timing=True)

start.record()
_ = torch.matmul(x, x)
end.record()

torch.cuda.synchronize()  # wait for everything to finish running on the GPU

print(f"GPU time: {0.001 * start.elapsed_time(end):6.5f}s")  # milliseconds to seconds

### Dynamic Computation Graph and Backpropagation

One of the main reasons for using PyTorch in Deep Learning projects is that we can automatically get **gradients/derivatives** of functions that we define. We will mainly use PyTorch for implementing neural networks, and they are just fancy functions. If we use weight matrices in our function that we want to learn, then those are called the **parameters** or simply the **weights**.

Given an input $\mathbf{x}$, we define our function by **manipulating** that input, usually by matrix-multiplications with weight matrices and additions with so-called bias vectors. As we manipulate our input, we are automatically creating a **computational graph**. This graph shows how to arrive at our output from our input.

PyTorch is a **define-by-run** framework; this means that we can just do our manipulations, and PyTorch will keep track of that graph for us. Thus, we create a dynamic computation graph along the way.

So, to recap: the only thing we have to do is to compute the **output**, and then we can ask PyTorch to automatically get the **gradients**.

> **Note:  Why do we want gradients?** Consider that we have defined a function, a neural net, that is supposed to compute a certain output $y$ for an input vector $\mathbf{x}$. We then define an **error measure** that tells us how wrong our network is; how bad it is in predicting output $y$ from input $\mathbf{x}$. Based on this error measure, we can use the gradients to **update** the weights $\mathbf{W}$ that were responsible for the output, so that the next time we present input $\mathbf{x}$ to our network, the output will be closer to what we want.

The first thing we have to do is to specify which tensors require gradients. By default, when we create a tensor, it does not require gradients.

In [ ]:
x = torch.ones((3,))
print(x.requires_grad)

We can change this for an existing tensor using the function `requires_grad_()` (underscore indicating that this is a in-place operation). Alternatively, when creating a tensor, you can pass the argument `requires_grad=True` to most initializers we have seen above.

In [ ]:
x.requires_grad_(True)
print(x.requires_grad)

In order to get familiar with the concept of a computation graph, we will create one for the following function:

$$y(x_i) = \frac{1}{N}\sum_{i=1}^N\left[(x_i + 2)^2 + 3\right]$$

where $N$ denotes the number of elements in $\mathbf{x}$. You could imagine that $\mathbf{x}$ are our parameters, and we want to optimize (either maximize or minimize) the output $y$. For this, we want to obtain the gradients $\partial y / \partial \mathbf{x}$. For our example, we'll use $\mathbf{x}=[0,1,2]$ as our input.

In [ ]:
x = torch.arange(3, dtype=torch.float32, requires_grad=True)  # only float tensors can have gradients
print(f"X:\n{x}")

Now let's build the computation graph step by step. You can combine multiple operations in a single line, but we will separate them here to get a better understanding of how each operation is added to the computation graph.

In [ ]:
a = x + 2
print("a:", a)
b = a ** 2
print("b:", b)
c = b + 3
print("c:", c)
y = c.mean()
print("Y:", y)

Using the statements above, we have created a computation graph that looks similar to the figure below:



We can perform backpropagation on the computation graph by calling the function `backward()` on the last output, which effectively calculates the gradients for each tensor that has the property `requires_grad=True`:

In [ ]:
y.backward()

`x.grad` will now contain the gradient $\partial y/ \partial \mathbf{x}$, and this gradient indicates how a change in $\mathbf{x}$ will affect output $y$ given the current input $\mathbf{x}=[0,1,2]$:

In [ ]:
print(x.grad)

We can also verify these gradients by hand. We will calculate the gradients using the chain rule, in the same way as PyTorch did it:

$$\frac{\partial y}{\partial x_i} = \frac{\partial y}{\partial c_i}\frac{\partial c_i}{\partial b_i}\frac{\partial b_i}{\partial a_i}\frac{\partial a_i}{\partial x_i}$$

Note that we have simplified this equation to index notation, and by using the fact that all operation besides the mean do not combine the elements in the tensor. Since

$$
a_i=x_i+2, \quad
b_i=a_i^2, \quad
c_i=b_i+3, \quad
y=\frac{1}{N}\sum_{i=1}^N c_i
$$

The partial derivatives are:

$$
\frac{\partial a_i}{\partial x_i} = 1, \hspace{1cm}
\frac{\partial b_i}{\partial a_i} = 2\cdot a_i, \hspace{1cm}
\frac{\partial c_i}{\partial b_i} = 1, \hspace{1cm}
\frac{\partial y}{\partial c_i} = \frac{1}{3}
$$

Hence $\partial y/\partial x_i=\frac{2}{3}(x_i+2)$.

Thus, being the input $\mathbf{x}=[0,1,2]$, our gradients are $\partial y/\partial \mathbf{x}=[4/3,2,8/3]$. The previous code cell should have printed the same result.

## Let's build our first Neural Network!

We are now going to create a simple neural network that classifies images from the [Street View House Numbers (SVHN) dataset](http://ufldl.stanford.edu/housenumbers/):

![](https://miro.medium.com/v2/resize:fit:1200/1*X-VO9sZXIaHxh3aF-CRXZg.png)

SVHN contains 600K 32x32 RGB images of printed digits (from 0 to 9) cropped from pictures of house number plates.

### Prepare the data

Download the dataset:

In [ ]:
tsfms = T.Compose([
    T.ToTensor(),
    T.Lambda(lambda x: x.flatten())
])

train_dset = datasets.SVHN(
    root="/data/",
    split="train",
    transform=tsfms,
    download=True
)
test_dset = datasets.SVHN(
    root="/data/",
    split="test",
    transform=tsfms,
    download=True
)

n_classes = 10

input_dim = len(train_dset[0][0])

In [ ]:
_, ax = plt.subplots(1, 4)

for i, idx in enumerate([0, 10, 20, 30]):
    img, label = test_dset[idx]
    ax[i].imshow(img.reshape(3, 32, 32).permute(1, 2, 0))
    ax[i].set_title(label)
    ax[i].axis("off")

Create a small validation set from the training set:

In [ ]:
train_subdset, val_subdset = random_split(
    train_dset,
    [len(train_dset) - 5000, 5000]
)

#### The data loader class

The class `torch.utils.data.DataLoader` represents a Python iterable over a dataset with support for automatic batching, multi-process data loading and many more features. The data loader communicates with the dataset using the `__getitem__` function, and stacks its outputs as tensors over the first dimension to form a batch.
In contrast to the dataset class, we usually don't have to define our own data loader class, but can just create an object of it with the dataset as input. Additionally, we can configure our data loader with the following input arguments (only a selection, see full list [here](https://pytorch.org/docs/stable/data.html#torch.utils.data.DataLoader)):

* `batch_size`: number of samples to stack per batch.
* `shuffle`: if True, the data is returned in a random order. This is important during training for introducing stochasticity.
* `num_workers`: number of subprocesses to use for data loading. The default, 0, means that the data will be loaded in the main process which can slow down training for datasets where loading a data point takes a considerable amount of time (e.g. large images). More workers are recommended for those, but can cause issues on Windows computers. For tiny datasets as ours, 0 workers are usually faster.
* `pin_memory`: if True, the data loader will copy tensors into CUDA pinned memory before returning them. This can save some time for large data points on GPUs. Usually a good practice to use for a training set, but not necessarily for validation and test to save memory on the GPU.
* `drop_last`: If True, the last batch is dropped in case it is smaller than the specified batch size. This occurs when the dataset size is not a multiple of the batch size. Only potentially helpful during training to keep a consistent batch size.

Let's create a simple data loader below:

In [ ]:
batch_size = 256

train_dl = DataLoader(
    train_subdset,
    batch_size=batch_size,
    shuffle=True
)
val_dl = DataLoader(
    val_subdset,
    batch_size=batch_size
)
test_dl = DataLoader(
    test_dset,
    batch_size=batch_size
)

In [ ]:
# next(iter(...)) catches the first batch of the data loader
# If shuffle is True, this will return a different batch every time we run this cell
# For iterating over the whole dataset, we can simple use "for batch in data_loader: ..."
data_inputs, data_labels = next(iter(train_dl))

# The shape of the outputs are [batch_size, d_1,...,d_N] where d_1,...,d_N are the
# dimensions of the data point returned from the dataset class
print(f"Data inputs: {data_inputs.shape}\n{data_inputs}")
print(f"\nData labels: {data_labels.shape}\n{data_labels}")

### Create the network

We will then create our first neural network. One way to create neural networks in PyTorch is by subclassing `torch.nn.Module`. In this way, our model will inherit a lot of ready-to-use convinience functions (access to parameters for optimization, get/set parameters, etc.).

We only need to create the layers we will use in the `__init__` function and define the `forward` function that specifies how to apply them.

Layers are in turn subclasses of `torch.nn.Module`. In our example, we will use only linear layers, i.e. Fully Connected (FC) layers. Our network will have at least two FC layers: `self.first`, mapping the flattened input image into the (first) hidden representation, and `self.last`, mapping the (last) hidden representation into the scores for the classes. `n_hidden_layers` specifies how many hidden layers our network has (`self.first` and `self.last` not included).


Note that to store a variable number of layers in our network we do not use plain Python lists, but `torch.nn.ModuleList`. This is important to make PyTorch aware of the layers in the list, e.g. to set/get their parameters when calling the methods of the base `Module` class.

In [ ]:
class SimpleClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, n_classes, n_hidden_layers=0):
        super().__init__()

        # Initialize the modules we need to build the network
        self.first = nn.Linear(input_dim, hidden_dim)
        self.activation = nn.ReLU()
        self.last = nn.Linear(hidden_dim, n_classes)

        self.hidden_layers = nn.ModuleList([
            nn.Linear(hidden_dim, hidden_dim) for i in range(n_hidden_layers)
        ])

    def forward(self, x):
        # Perform the calculation of the model to determine the prediction
        x = self.first(x)
        x = self.activation(x)
        for layer in self.hidden_layers:
            x = layer(x)
            x = self.activation(x)
        x = self.last(x)

        return x

In [ ]:
hidden_dim = 128

model = SimpleClassifier(
    input_dim,
    hidden_dim,
    n_classes,
    n_hidden_layers=1
)
for name, params in model.named_parameters():
    print(f"{name}: {params.shape}")

Each linear layer has a weight matrix of shape `[output, input]`, and a bias of the shape `[output]`. The activation function does not have any parameters. Note that parameters are only registered for `nn.Module` objects that are direct object attributes, i.e. `self.a = ...`.

`state_dict()` instead is an (ordered) dictionary, which associates each variable storing a layer with its parameters. It is therefore useful to obtain a snapshot of the parameters of our model that can later be restored by calling `load_state_dict()`.

In [ ]:
type(model.state_dict())

In [ ]:
model.state_dict().keys()

In [ ]:
model.state_dict()["first.bias"]

In [ ]:
# Push model to device. Has to be done only once.
model.to(device)

### Optimization

After defining the model and the dataset, it is time to prepare the optimization of the model. During training, we will perform the following steps:

1. Get a batch from the data loader
```
for (x,y) in dataset:
```
2. Obtain the predictions from the model for the batch
```
y_pred = model(x)
```
3. Calculate the loss based on the difference between predictions and labels
```
loss = fcn(y_pred, y)
```
4. Backpropagation: calculate the gradients for every parameter with respect to the loss
```
optimizer.zero_grad()
loss.backward()
```
5. Update the parameters of the model in the direction of the gradients
```
optimizer.step()
```
To update the parameters, PyTorch provides the package `torch.optim` that has most popular optimizers implemented. We will for now use `torch.optim.Adam`.

In [ ]:
optimizer = Adam(model.parameters(), lr=0.001)

The optimizer provides two useful functions: `optimizer.step()` and `optimizer.zero_grad()`. The step function updates the parameters based on the gradients as explained above.

The function `optimizer.zero_grad()` sets the gradients of all parameters to zero. While this function seems less relevant at first, it is a crucial pre-step before performing backpropagation. If we call the `backward` function on the loss while the parameter gradients are non-zero from the previous batch, the new gradients will actually be added to the previous ones instead of overwriting them. This is done because a parameter might occur multiple times in a computation graph, and we need to sum the gradients in this case instead of replacing them. Hence, remember to call `optimizer.zero_grad()` before calculating the gradients of a batch.

In [ ]:
def ncorrect(scores, y):
    y_hat = torch.argmax(scores, -1)
    return (y_hat == y).sum()

def accuracy(scores, y):
    correct = ncorrect(scores, y)
    return correct.true_divide(y.shape[0])

def train_loop(model, train_dl, epochs, opt, val_dl=None, verbose=False):
    best_val_acc = 0
    best_params = []
    best_epoch = -1

    for e in tqdm(range(epochs)):
        # ----------[ Training loop ]---------- #
        model.train()
        train_loss = 0
        train_samples = 0
        train_acc = 0

        for train_data in train_dl:
            imgs = train_data[0].to(device)
            labels = train_data[1].to(device)

            scores = model(imgs)

            loss = F.cross_entropy(scores, labels)
            train_loss += loss.item() * imgs.shape[0]
            train_samples += imgs.shape[0]
            train_acc += ncorrect(scores, labels).item()

            opt.zero_grad()  # clear
            loss.backward()  # fill
            opt.step()       # use

        train_acc /= train_samples
        train_loss /= train_samples

        # ----------[ Validation loop ]---------- #
        model.eval()
        with torch.no_grad():
            val_loss = 0
            val_samples = 0
            val_acc = 0

            if val_dl is not None:
                for val_data in val_dl:
                    imgs = val_data[0].to(device)
                    labels = val_data[1].to(device)

                    val_scores = model(imgs)

                    val_loss += F.cross_entropy(val_scores, labels).item() * imgs.shape[0]
                    val_samples += imgs.shape[0]
                    val_acc += ncorrect(val_scores, labels).item()
                val_acc /= val_samples
                val_loss /= val_samples

            if val_dl is None or val_acc > best_val_acc:
                best_val_acc = val_acc if val_dl is not None else 0
                best_params = model.state_dict()
                torch.save(best_params, "best_model.pth")
                best_epoch = e

        if verbose and e % 5 == 0:
            print(f"Epoch {e}: train loss {train_loss:.3f} - train acc {train_acc:.3f}" + ("" if val_dl is None else f" - valid loss {val_loss:.3f} - valid acc {val_acc:.3f}"))

    if verbose and val_dl is not None:
        print(f"Best epoch {best_epoch}, best acc {best_val_acc}")

    return best_val_acc, best_params, best_epoch

### Train the network

In [ ]:
epochs = 20

best_val_acc, best_params, best_epoch = train_loop(
    model,
    train_dl,
    epochs,
    optimizer,
    val_dl,
    verbose=True
)

### Load the model

In [ ]:
# Load state dict from the disk (make sure it is the same name as above)
state_dict = torch.load("best_model.pth")

# Create a new model and load the state dict
new_model = model = SimpleClassifier(
    input_dim,
    hidden_dim,
    n_classes,
    n_hidden_layers=1
)
new_model.load_state_dict(state_dict)

# Now it's your turn!

## Exercise 1: improve the accuracy

Try to achieve a better accuracy on the validation set by modifying the previous model and manually tuning the hyper-parameters such as the number of epochs, the learning rate, and the number of hidden layers.


Also, add [Batch Normalization](https://arxiv.org/abs/1502.03167). You can find the normalization layers already implemented in PyTorch [here](https://pytorch.org/docs/stable/nn.html#normalization-layers). Use `torch.nn.Sequential` to stack together a block with the following
structure: FC $\rightarrow$ BatchNorm $\rightarrow$ Activation.

You should easily get 80% accuracy on the validation set.


In [ ]:
def get_linear_bn_relu(input_dim, out_dim):
    """Return a Sequential module with a sandwich of Linear, Batch Norm, and ReLU.

    Args:
        input_dim: the number of input features.
        out_dim: the number of the output features.

    Returns: the created Sequential block.
    """
    modules = []
    # ==================================================================== #
    #                         YOUR CODE STARTS HERE                        #
    # ==================================================================== #

    # ==================================================================== #
    #                         YOUR CODE ENDS HERE                          #
    # ==================================================================== #

    return nn.Sequential(*modules)

In [ ]:
class SimpleClassifierBN(nn.Module):
    def __init__(self, input_dim, hidden_dim, n_classes, n_hidden_layers=0):
        super().__init__()

        self.first = get_linear_bn_relu(input_dim, hidden_dim)
        self.last = nn.Linear(hidden_dim, n_classes)

        self.hidden_layers = nn.ModuleList([
            get_linear_bn_relu(hidden_dim, hidden_dim) for i in range(n_hidden_layers)
        ])

    def forward(self, x):
        x = self.first(x)
        for layer in self.hidden_layers:
            x = layer(x)
        x = self.last(x)

        return x

In [ ]:
# ==================================================================== #
#                         YOUR CODE STARTS HERE                        #
# ==================================================================== #

# ==================================================================== #
#                         YOUR CODE ENDS HERE                          #
# ==================================================================== #

best_val_acc, best_params, best_epoch = train_loop(
    new_model,
    train_dl,
    epochs,
    optimizer,
    val_dl,
    verbose=True)

In [ ]:
new_model.load_state_dict(torch.load("best_model.pth"))

## Exercise 2: confusion matrix
Compute the confusion matrix for the test set. Then, from the confusion matrix, compute the accuracy score.

> **Note:** a confusion matrix $\mathbf{C}$ is such that $\mathbf{C}_{ij}$ is equal to the number of samples known to belong to class $i$ and predicted to belong to class $j$.

In [ ]:
import seaborn as sn
import pandas as pd

In [ ]:
new_model.eval()
confusion_matrix = torch.zeros(n_classes, n_classes, dtype=torch.int)
# ==================================================================== #
#                         YOUR CODE STARTS HERE                        #
# ==================================================================== #

# ==================================================================== #
#                         YOUR CODE ENDS HERE                          #
# ==================================================================== #
print(test_acc)

In [ ]:
conf_mtrx_df = pd.DataFrame(confusion_matrix)
sn.heatmap(conf_mtrx_df, annot=True, fmt="d");

## Exercise 3: nearest neighbor search

Rewrite the `forward` method of the classifier class to return the feature vector produced by the network right before the classification scores.

> **Note:** you don't have to retrain the model again!

Then, show the three nearest neighbors in the training set for element with index 0 in the test set.

In [ ]:
class SimpleClassifierBNv2(nn.Module):
    def __init__(self, input_dim, hidden_dim, n_classes, n_hidden_layers=0):
        super().__init__()

        # ==================================================================== #
        #                         YOUR CODE STARTS HERE                        #
        # ==================================================================== #

        # ==================================================================== #
        #                         YOUR CODE ENDS HERE                          #
        # ==================================================================== #

    def forward(self, x):
        # ==================================================================== #
        #                         YOUR CODE STARTS HERE                        #
        # ==================================================================== #

        # ==================================================================== #
        #                         YOUR CODE ENDS HERE                          #
        # ==================================================================== #

In [ ]:
state_dict = torch.load("best_model.pth")

new_model = SimpleClassifierBNv2(
    input_dim,
    hidden_dim,
    n_classes,
    n_hidden_layers=3
)
new_model.load_state_dict(state_dict)
new_model.to(device)
new_model.eval();

In [ ]:
def get_feat_vecs(model, dl):
    feat_vecs = []
    labels = []
    # ==================================================================== #
    #                         YOUR CODE STARTS HERE                        #
    # ==================================================================== #

    # ==================================================================== #
    #                         YOUR CODE ENDS HERE                          #
    # ==================================================================== #

    return feat_vecs, labels

train_dl = DataLoader(
    train_subdset,
    batch_size=batch_size,
    shuffle=False  # no shuffle here
)
feat_vecs_train, labels_train = get_feat_vecs(new_model, train_dl)
feat_vecs_test, labels_test = get_feat_vecs(new_model, test_dl)

In [ ]:
test_idx = 0

# ==================================================================== #
#                         YOUR CODE STARTS HERE                        #
# ==================================================================== #

# ==================================================================== #
#                         YOUR CODE ENDS HERE                          #
# ==================================================================== #